First let's load the Jar. This requires `mill q.assembly` to be run beforehand.

In [1]:
interp.load.cp(os.pwd/".."/"out"/"q"/"assembly.dest"/"out.jar")

We will load some boilerplate code for graphing as well.

In [2]:
import $file.makeplot

import $file.$

Now, all the imports...

In [3]:
import riscq._
import riscq.tester.qubic._
import riscq.soc.QubicSoc
import riscq.tester._
import spinal.core.sim._
import spinal.core._
import spinal.lib.bus.tilelink.sim._
import spinal.lib.bus.tilelink._
import spinal.lib.bus.amba4.axi.sim.Axi4Master
import riscq.tester.RvAssembler
import riscq.tester.QubicAssembler
import riscq.tester.ByteHelper
import riscq.soc.QubicPlugins
import QubicTestConfig._
import scala.collection.mutable.ArrayBuffer
import spinal.lib._
import riscq.soc.MemoryMapSoc
import riscq.tester.RvAssembler
import riscq.tester.mmsoc.MMSocDriver

import riscq._
import riscq.tester.qubic._
import riscq.soc.QubicSoc
import riscq.tester._
import spinal.core.sim._
import spinal.core._
import spinal.lib.bus.tilelink.sim._
import spinal.lib.bus.tilelink._
import spinal.lib.bus.amba4.axi.sim.Axi4Master
import riscq.tester.RvAssembler
import riscq.tester.QubicAssembler
import riscq.tester.ByteHelper
import riscq.soc.QubicPlugins
import QubicTestConfig._
import scala.collection.mutable.ArrayBuffer
import spinal.lib._
import riscq.soc.MemoryMapSoc
import riscq.tester.RvAssembler
import riscq.tester.mmsoc.MMSocDriver

First, let's copy over the existing QubicSOC test and collect some output. This skips hand-writing assembly for the moment.

In [ ]:
val dac_output = ArrayBuffer[Double]()
val carrier_output = ArrayBuffer[Double]()
simConfig
    .compile{
      val dut = QubicSoc(
        qubitNum = 8,
        withVivado = false,
        withCocotb = false,
        withWhitebox = true,
        withTest = true
      )
      dut.pgs.map{_.io.simPublic()}
      dut.cgs.map{_.io.simPublic()}
      dut
    }
    .doSim { dut =>
      val driver = new Driver(dut)
      import driver._
      val rvAsm = new RvAssembler(128)
      import rvAsm._
      val qbAsm = new QubicAssembler()
      import qbAsm._

      init()

      val batchSize = 16
      val dataWidth = 16
      for (i <- 0 until 5) {
        val batch = List.tabulate(batchSize)((x : Int) => BigInt(200 * x * (i + 1))).reverse
        val batchData = riscq.pulse.PGTestPulse.concat(batch, dataWidth)
        val dataStr = batch.map { x => ByteHelper.intToBinStr(x, dataWidth) }.reduce { _ ++ _ }
        tlDriver.putFullData(0, dut.pulseOffset + i * batchSize * dataWidth / 8, ByteHelper.fromBinStr(dataStr).reverse)
      }
      cd100m.waitRisingEdge()

      val startTime = 50
      val insts = List(
        setTime(0), // 0
        carrier(1 << (16 - 5), 0), // 1
        qbAsm.pulse(
          puop,
          start = startTime,
          addr = 0,
          duration = 4,
          phase = (1 << (puop.phaseWidth - 7)),
          freq = 0,
          amp = (1 << (puop.ampWidth - 1)) - 1
        ), // 2
        beq(0, 0, 0) // 3
      )
      loadIMem(0, insts)

      dut.riscq_rst #= true
      tick()
      dut.riscq_rst #= false

      tick(startTime+8)
      for (i <- 0 until 6) {
        logTime()
        logPcs()
        logDac(0)
        logCarrier(0)
        println("")
        val dac = dut.pgs(0).io.data
        val pulse = dac.payload.map { _.r.toDouble * (1 << 14) }.toList
        dac_output ++= pulse
        val carrier = dut.cgs(0).io.carrier
        val data = carrier.payload.map { _.r.toDouble * (1 << 14) }.toList
        carrier_output ++= data
        tick()
      }
    }

Now, let's graph the DAC and Carrier data.

In [ ]:
val pngBytes = makeplot.make_plot(List(makeplot.LabeledData("DAC output", dac_output), makeplot.LabeledData("Carrier output", carrier_output)))
Image(pngBytes)

Now, let's fix the envelope and do the same!

In [ ]:
// TODO

And now we write some assembly and try the same with MMSoc too. TODO for this to work, need to run some build script in the "asm" folder.

In [ ]:
object TestAlu extends App {
  val simConfig = SimConfig.addSimulatorFlag("-Wno-MULTIDRIVEN")
  simConfig.compile(MemoryMapSoc(withWhitebox = true)).doSim { dut =>
    val driver = new MMSocDriver(dut)
    driver.loadInsts()
    driver.init()
    driver.rstUp()

    dut.riscq_rst #= true
    for(_ <- 0 until 15) {
      driver.tick()
    }

    driver.rstDown()
    dut.riscq_rst #= false
    for(_ <- 0 until 100) {
      driver.logRf()
      driver.tick()
    }
  }
}

Graph it too.